In [84]:
%pip install flatten_json pymongo pandas sklearn
import pandas as pd
import pymongo
import sklearn
from flatten_json import flatten


Note: you may need to restart the kernel to use updated packages.


In [85]:
mongo_client = pymongo.MongoClient("mongodb://localhost:27017/")
mongo_database = mongo_client["tft_data"]
mongo_container = mongo_database["challenger"]
all_data = list(mongo_container.find().limit(100))

In [86]:
df = pd.json_normalize(all_data, ["info", "participants"], ["metadata"])

def trait_object_array_to_trait_array(trait_object_array):
   return [f"{trait['name']}_{trait['num_units']}_units" for trait in trait_object_array]

def units_object_array_to_unit_array(unit_object_array):
   unit_array = []  
   for unit in unit_object_array:
      items = ""
      for index, item in enumerate(unit['itemNames']):
         items += f"+item{index + 1}{item.replace('TFT_Item', '')}"
      unit_array.append(f"tier_{unit['tier']}_{unit['character_id']}{items}")
   return unit_array

def prefix_augements_with_order(augments):
   return [f"{index + 1}_{augment}" for index, augment in enumerate(augments)]

def get_match_id(metadata):
   return metadata['match_id']

df['traits'] = df['traits'].apply(trait_object_array_to_trait_array)
df['units'] = df['units'].apply(units_object_array_to_unit_array)
df['match_id'] = df['metadata'].apply(get_match_id)
df['augments'] = df['augments'].apply(prefix_augements_with_order)

from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer(sparse_output=True)

df = df.join(
            pd.DataFrame.sparse.from_spmatrix(
                mlb.fit_transform(df.pop('augments')),
                index=df.index,
                columns=mlb.classes_)).join(
            pd.DataFrame.sparse.from_spmatrix(
                mlb.fit_transform(df.pop('traits')),
                index=df.index,
                columns=mlb.classes_)).join(
            pd.DataFrame.sparse.from_spmatrix(
                mlb.fit_transform(df.pop('units')),
                index=df.index,
                columns=mlb.classes_))


df = df.drop(columns=['metadata', 'traits', 'units'])

df.to_csv("out.csv", index=False)
df

/tmp/ipykernel_42551/1217064332.py:47: FutureWarning: The behavior of .astype from SparseDtype to a non-sparse dtype is deprecated. In a future version, this will return a non-sparse array with the requested dtype. To retain the old behavior, use `obj.astype(SparseDtype(dtype))`
  df.to_csv("out.csv", index=False)


KeyboardInterrupt: 